# Imagenet 数据管理
* ResNet50分类测试集
* 去除子目录中文名
* 添加子目录中文名

## ResNet50分类测试集

In [3]:
import os
import numpy as np
import h5py
import tensorflow as tf
import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from mylibs.ProcessBar import ShowProcess

#---------参数信息---------
test_path ='%s/e/dataset_tiptical/image_net/ILSVRC2012_img_test'%os.getenv('HOME') #测试文件夹
preds_path='%s/work/temp/temp1'%os.getenv('HOME')          #软链接输出目录

#---------加载模型---------
#加载模型
model=ResNet50(weights='imagenet')
#打印模型
model.summary()

#---------测试集分类-------
pb = ShowProcess(100,'','', 'OK') #进度条
#逐个文件测试
items=os.listdir(test_path)
count=len(items)
for i,sfile in enumerate(items):
    img_path=os.path.join(test_path,sfile)                      #图像完整路径
    if not os.path.isdir(img_path):
        img = image.load_img(img_path, target_size=(224, 224)) #加载图像
        x = image.img_to_array(img)    #to_array
        x = np.expand_dims(x, axis=0)  #扩展batch_size维度
        x = preprocess_input(x)        #图像预处理
        preds = model.predict(x)       #模型预测
        #print('Predicted:', decode_predictions(preds, top=3)[0])
        #       Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]
        top3=decode_predictions(preds,top=3)[0]  #解码top3结果
        wnid=top3[0][0]                           #WNID
        #创建软链接
        src='%s/%s'%(test_path,sfile)
        dst_path='%s/%s'%(preds_path,wnid)
        dst='%s/%s'%(dst_path,sfile)
        os.makedirs(dst_path) if not os.path.exists(dst_path) else ''
        os.symlink(src,dst)
        if i % 20 == 0:
            pb.show_process(int(i*100./count),'','%d/%d'%(i,count))
pb.show_process(100)
print('Done!')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

KeyboardInterrupt: 

## 去除子目录中文名

In [3]:
import os
import shutil
import numpy as np
from mylibs.data.ImageNetData import ImageNetData

#ImageNet 配置目录
imageNet_data_path='%s/e/dataset_tiptical/image_net/ILSVRC2012_devkit_t12/data'%os.getenv('HOME')
#测试数据集目录
data_path ='%s/e/dataset_tiptical/image_net/ILSVRC2012_img_test_wnid'%os.getenv('HOME') #数据目录

#加载ImageNetData
dataInfo=ImageNetData(imageNet_data_path)

#去掉子目录中文名
dataInfo.rm_cn_name(data_path)

## 添加子目录中文名

In [2]:
import os
import numpy as np
from mylibs.data.ImageNetData import ImageNetData

#ImageNet 配置目录
imageNet_data_path='%s/e/dataset_tiptical/image_net/ILSVRC2012_devkit_t12/data'%os.getenv('HOME')
#测试数据集目录
data_path ='%s/e/dataset_tiptical/image_net/ILSVRC2012_img_test_wnid'%os.getenv('HOME') #数据目录

#加载ImageNetData
dataInfo=ImageNetData(imageNet_data_path)
#添加子目录中文名
dataInfo.add_cn_name(data_path)